In [15]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv(r'D:\IMP  ML  PROJECTS\CAR PRICE PREDICTION\web scraping\MUMBAI\feature engineering\cleaned_car_data_2026_mumbai.csv')

In [17]:
df.columns

Index(['Registration Year', 'Insurance', 'Fuel Type', 'Seats', 'Kms Driven',
       'Ownership', 'Engine Displacement', 'Transmission', 'Power',
       'Drive Type', 'Mileage', 'Engine Type', 'Max_Power_clean',
       'Max_Power_RPM', 'Max_Torque_Nm', 'Max_Torque_RPM', 'No. of Cylinders',
       'Valves Per Cylinder', 'Fuel Supply System', 'Turbo Charger',
       'Transmission Type', 'Gearbox', 'Petrol Mileage ARAI',
       'Petrol Fuel Tank Capacity', 'Emission Norm Compliance',
       'Front Suspension', 'Rear Suspension', 'Steering Type',
       'Steering Column', 'Front Brake Type', 'Rear Brake Type', 'Length',
       'Width', 'Height', 'Ground Clearance Unladen', 'Wheel Base',
       'Gross Weight', 'Price', 'Diesel Mileage ARAI',
       'Diesel Fuel Tank Capacity', 'Top Speed', 'Super Charge',
       'Battery Capacity', 'Charging Time DC', 'Charging Time AC',
       'Motor Type', 'Battery Type', 'Charging Port'],
      dtype='object')

In [18]:
# Split the df over the 'Fuel_Type' column and make new dfs

for i in df['Fuel Type'].unique():
    globals()[i] = df[df['Fuel Type'] == i]

    # print the result 
    print(globals()[i].head())
    print("\n")
    

   Registration Year      Insurance Fuel Type  Seats  Kms Driven  Ownership  \
0             2022.0        Unknown    Petrol    5.0       30000          1   
2             2024.0  Comprehensive    Petrol    7.0        8000          1   
3             2024.0        Unknown    Petrol    7.0       20277          1   
4             2022.0  Comprehensive    Petrol    5.0       20000          1   
5             2023.0  Comprehensive    Petrol    5.0       27802          1   

   Engine Displacement Transmission       Power Drive Type  ...  \
0                  999    Automatic  113.98 bhp        FWD  ...   
2                 1451    Automatic  141.04 bhp        FWD  ...   
3                 1482    Automatic  157.81 bhp        FWD  ...   
4                  999       Manual   71.01 bhp        FWD  ...   
5                 1497    Automatic         NaN        2WD  ...   

   Diesel Mileage ARAI Diesel Fuel Tank Capacity  Top Speed  Super Charge  \
0       Not Applicable            Not Applica

### <span style="color: yellow;">Working over the 'globals()['Petrol]" dataset</span>

In [19]:
globals()['Petrol'].columns
print("Number of columns initially : ", len(globals()['Petrol'].columns))

Number of columns initially :  48


In [ ]:
del_cols = [
    'Diesel Mileage ARAI', 'Diesel Fuel Tank Capacity', 'Super Charge',
       'Battery Capacity', 'Charging Time DC', 'Charging Time AC',
       'Motor Type', 'Battery Type', 'Charging Port', 'Fuel Type'
]

globals()['Petrol'] = globals()['Petrol'].drop(del_cols, axis=1)
print("Number of columns after cleaning : ", len(globals()['Petrol'].columns))

Number of columns after cleaning :  39


In [22]:
globals()['Petrol'].columns

Index(['Registration Year', 'Insurance', 'Fuel Type', 'Seats', 'Kms Driven',
       'Ownership', 'Engine Displacement', 'Transmission', 'Power',
       'Drive Type', 'Mileage', 'Engine Type', 'Max_Power_clean',
       'Max_Power_RPM', 'Max_Torque_Nm', 'Max_Torque_RPM', 'No. of Cylinders',
       'Valves Per Cylinder', 'Fuel Supply System', 'Turbo Charger',
       'Transmission Type', 'Gearbox', 'Petrol Mileage ARAI',
       'Petrol Fuel Tank Capacity', 'Emission Norm Compliance',
       'Front Suspension', 'Rear Suspension', 'Steering Type',
       'Steering Column', 'Front Brake Type', 'Rear Brake Type', 'Length',
       'Width', 'Height', 'Ground Clearance Unladen', 'Wheel Base',
       'Gross Weight', 'Price', 'Top Speed'],
      dtype='object')